<a href="https://colab.research.google.com/github/MansiChache/Yojana_Guide/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain transformers faiss-cpu llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 12.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.6-cp310-cp310-linux_x86_64.whl size=4070569 sha256=2c44b4197b15e300d4d58c742bbf23dd765d1306f0567956ffbde7e65cd744b5
  Stored in directory: /root/.cache/pip/wheels/06/61/7b/d5ebacdccd230f42c847b6eec707a26c8455f49277393b295c
Successfully built llama-cpp-python


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain_community
!pip install transformers sentencepiece
!pip install autoawq
!pip install peft

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig
import json
import os

# Model & Data Paths
base_model_path = "/content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/model"
data_path = "/content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/schemes_Data/training_data.json"
output_dir = "/content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/finetuned_model"

# Load training data
with open(data_path, "r") as f:
    data = json.load(f)

# Prepare training data
train_data = [
    f"Scheme Name: {item['scheme_name']}\nDetails: {item['details']}\nBenefits: {item['benefits']}\n"
    f"Eligibility: {item['eligibility']}\nApplication Process: {item['application_process']}\n"
    f"Documents Required: {item['documents_required']}\n\n"
    for item in data
]

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=False)

# Set pad_token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_function(examples):
    tokenized_input = tokenizer(examples, padding="max_length", truncation=True, max_length=256)
    tokenized_input["labels"] = tokenized_input["input_ids"]
    return tokenized_input

# Tokenize data
tokenized_data = [tokenize_function(item) for item in train_data]

# ✅ Load Base Model (First Time Training)
model = AutoModelForCausalLM.from_pretrained(base_model_path)

# ✅ Move Model to GPU (Fix CUDA issue)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Apply LoRA for efficient fine-tuning
lora_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, lora_config)

# ✅ Training arguments (Fixed `evaluation_strategy`)
training_args = TrainingArguments(
    output_dir=output_dir,  # Save finetuned model here
    num_train_epochs=2,  # Pehli baar ke liye 2 epochs kaafi h
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="no",  # 🔥 Fix: Disable evaluation since eval data is missing
    fp16=True,  # Mixed precision for faster training
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    tokenizer=tokenizer
)

# ✅ Fine-tune the model
trainer.train()

# ✅ Save the fine-tuned model
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Fine-tuned model saved at {output_dir}")


You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-e731a9dc13ad>:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kdmehta2503 (kdmehta2503-fr-conceicao-rodrigues-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,1.999700
20,1.896100
30,1.768400
40,1.711300
50,1.636900
60,1.593500
70,1.555200
80,1.528000
90,1.489700
100,1.475000


Step,Training Loss
10,1.999700
20,1.896100
30,1.768400
40,1.711300
50,1.636900
60,1.593500
70,1.555200
80,1.528000
90,1.489700
100,1.475000


Fine-tuned model saved at /content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/finetuned_model


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig
import json
import os
import torch
from datasets import Dataset

# ✅ Model & Data Paths
fine_tuned_model_path = "/content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/finetuned_model"
data_path = "/content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/schemes_Data/training_data.json"

# ✅ Load training data
with open(data_path, "r") as f:
    data = json.load(f)

# ✅ Prepare training data
train_texts = [
    f"Scheme Name: {item['scheme_name']}\nDetails: {item['details']}\nBenefits: {item['benefits']}\n"
    f"Eligibility: {item['eligibility']}\nApplication Process: {item['application_process']}\n"
    f"Documents Required: {item['documents_required']}\n"
    for item in data
]

# ✅ Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path, trust_remote_code=False)

# ✅ Set pad_token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ✅ Tokenization function with labels
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()  # ✅ Set labels to input_ids for loss computation
    return tokens

# ✅ Create dataset & tokenize
train_dataset = Dataset.from_dict({"text": train_texts}).map(tokenize_function, batched=True)

# ✅ Load previously fine-tuned model
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)

# ✅ Apply LoRA for efficient fine-tuning
lora_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, lora_config)

# ✅ Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir=fine_tuned_model_path,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="no",  # ❌ No evaluation dataset, so disable it
    fp16=True,  # Enable mixed precision for speed (if using GPU)
    save_total_limit=2,
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # ✅ Properly formatted dataset
    tokenizer=tokenizer
)

# ✅ Start training
trainer.train()

# ✅ Save the updated model
model.save_pretrained(fine_tuned_model_path)
tokenizer.save_pretrained(fine_tuned_model_path)

print(f"Model updated and saved at {fine_tuned_model_path}")



Map:   0%|          | 0/2163 [00:00<?, ? examples/s]

You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-5460929a136e>:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kdmehta2503 (kdmehta2503-fr-conceicao-rodrigues-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,1.999700
20,1.895500
30,1.767300
40,1.709900
50,1.634800
60,1.591700
70,1.552500
80,1.525500
90,1.486900
100,1.472400


Model updated and saved at /content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/finetuned_model


In [ ]:
import json

# ✅ Load training data (since validation data is same)
train_data_path = "/content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/schemes_Data/combined_schemes_data.json"
with open(train_data_path, "r") as f:
    train_data = json.load(f)

# ✅ Automatically generate questions from the dataset
generated_questions = []

for scheme in train_data:
    scheme_name = scheme["scheme_name"]

    # Generate 4 questions per scheme
    questions = [
        {"question": f"What is the eligibility for {scheme_name}?", "answer": scheme["eligibility"]},
        {"question": f"What are the benefits of {scheme_name}?", "answer": scheme["benefits"]},
        {"question": f"How can I apply for {scheme_name}?", "answer": scheme["application_process"]},
        {"question": f"What documents are needed for {scheme_name}?", "answer": scheme["documents_required"]}
    ]

    generated_questions.extend(questions)

# ✅ Save the generated questions to a new JSON file
val_data_path = "/content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/schemes_Data/generated_validation_data.json"
with open(val_data_path, "w") as f:
    json.dump(generated_questions, f, indent=4)

print(f"✅ Successfully generated {len(generated_questions)} validation questions!")


✅ Successfully generated 10816 validation questions!


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig
import json
import os
import torch
from datasets import Dataset

# ✅ Model & Data Paths
fine_tuned_model_path = "/content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/finetuned_model"
data_path = "/content/drive/MyDrive/Colab Notebooks/gov_schemes_chatbot_using_llm/schemes_Data/combined_schemes_data.json"

# ✅ Load training data
with open(data_path, "r") as f:
    data = json.load(f)

# ✅ Prepare training data
train_texts = [
    f"Scheme Name: {item['scheme_name']}\nDetails: {item['details']}\nBenefits: {item['benefits']}\n"
    f"Eligibility: {item['eligibility']}\nApplication Process: {item['application_process']}\n"
    f"Documents Required: {item['documents_required']}\n"
    for item in data
]

# ✅ Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path, trust_remote_code=False)

# ✅ Set pad_token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ✅ Tokenization function with labels
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()  # ✅ Set labels to input_ids for loss computation
    return tokens

# ✅ Create dataset & tokenize
train_dataset = Dataset.from_dict({"text": train_texts}).map(tokenize_function, batched=True)

# ✅ Load previously fine-tuned model
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)

# ✅ Apply LoRA for efficient fine-tuning
lora_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, lora_config)

# ✅ Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir=fine_tuned_model_path,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="no",  # ❌ No evaluation dataset, so disable it
    fp16=True,  # Enable mixed precision for speed (if using GPU)
    save_total_limit=2,
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # ✅ Properly formatted dataset
    tokenizer=tokenizer
)

# ✅ Start training
trainer.train()

# ✅ Save the updated model
model.save_pretrained(fine_tuned_model_path)
tokenizer.save_pretrained(fine_tuned_model_path)

print(f"Model updated and saved at {fine_tuned_model_path}")

Map:   0%|          | 0/2704 [00:00<?, ? examples/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-d839124b7edf>:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,2.028000
20,1.856100


KeyboardInterrupt: 